# Modelos de Difusión Texto-a-Imagen: El Enfoque de DALL-E y Stable Diffusion

## Objetivo de este Notebook

Este notebook explica cómo funcionan los modelos de difusión **texto-a-imagen** como DALL-E 2, Stable Diffusion e Imagen. Aprenderás los componentes clave que permiten generar imágenes a partir de descripciones textuales.

### ¿Qué aprenderás?

1. Arquitectura de modelos texto-a-imagen
2. Cómo funcionan los text encoders (CLIP, T5)
3. Conditional diffusion: guiar la generación con texto
4. Classifier-free guidance para mejorar calidad
5. Latent diffusion: generar en espacio comprimido (Stable Diffusion)
6. Implementación práctica usando Stable Diffusion

### Pre-requisitos

- Completar el notebook 01_diffusion_fundamentals.ipynb
- Conocimientos básicos de transformers y embeddings
- Familiaridad con PyTorch

---

## 1. Introducción: De Diffusion a Text-to-Image

### Evolución de los Modelos

1. **Diffusion Models básicos** (2020): Generan imágenes desde ruido
   - No hay control sobre qué se genera
   - Ej: DDPM de Ho et al.

2. **Conditional Diffusion** (2021): Añade control mediante condiciones
   - Puede ser guiado por clases, layouts, etc.
   - Ej: Guided Diffusion

3. **Text-to-Image Diffusion** (2022): Condicionado por texto natural
   - Usa text encoders poderosos (CLIP, T5)
   - Ej: DALL-E 2, Imagen, Stable Diffusion

4. **Latent Diffusion** (2022): Opera en espacio latente comprimido
   - Mucho más eficiente
   - Ej: Stable Diffusion

### Arquitectura General

```
Texto → [Text Encoder] → Text Embeddings → [U-Net Condicionada] → Imagen
                            ↓
                    (guía el denoising)
```

## 2. Instalación e Importación

Para este notebook, usaremos la librería `diffusers` de Hugging Face, que proporciona implementaciones de modelos de difusión pre-entrenados.

In [1]:
# Instalar dependencias
!pip install diffusers transformers accelerate torch torchvision matplotlib pillow

  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nv

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from diffusers import StableDiffusionPipeline, DDPMScheduler, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Usando dispositivo: cpu


## 3. Componente Clave 1: Text Encoder (CLIP)

### ¿Qué es CLIP?

CLIP (Contrastive Language-Image Pre-training) es un modelo que aprende a asociar texto e imágenes. Fue entrenado con 400 millones de pares (imagen, texto) de internet.

**Características clave:**
- Convierte texto en embeddings de 768 dimensiones (CLIP-ViT-L/14)
- Los embeddings capturan el significado semántico
- Textos similares tienen embeddings cercanos en el espacio latente

### ¿Cómo se usa en Diffusion?

```
"Un gato en un sombrero" → [CLIP Text Encoder] → [emb₁, emb₂, ..., emb_n]
                                                          ↓
                                              [U-Net condicionada]
                                                          ↓
                                                  Imagen del gato
```

In [ ]:
# Cargar CLIP text encoder
print("Cargando CLIP text encoder...")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

# Función para obtener embeddings de texto
def get_text_embeddings(prompts):
    """
    Convierte texto en embeddings usando CLIP
    
    Args:
        prompts: Lista de strings o un solo string
    
    Returns:
        Text embeddings [batch_size, seq_len, hidden_dim]
    """
    if isinstance(prompts, str):
        prompts = [prompts]
    
    # Tokenizar el texto
    text_input = tokenizer(
        prompts,
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt"
    )
    
    # Obtener embeddings
    with torch.no_grad():
        text_embeddings = text_encoder(text_input.input_ids.to(device))[0]
    
    return text_embeddings


# Ejemplo: Generar embeddings de diferentes prompts
prompts = [
    "A photo of a cat",
    "A photo of a dog",
    "A picture of a feline",  # Sinónimo de cat
    "An image of a canine"     # Sinónimo de dog
]

embeddings = get_text_embeddings(prompts)
print(f"\nShape de embeddings: {embeddings.shape}")
print(f"[batch_size, sequence_length, hidden_dimension]")

# Calcular similitud entre embeddings (pooled)
# Tomamos el embedding del token [CLS] o hacemos mean pooling
pooled_embeddings = embeddings.mean(dim=1)  # [batch_size, hidden_dim]

# Calcular similitud coseno
similarity_matrix = F.cosine_similarity(
    pooled_embeddings.unsqueeze(1), 
    pooled_embeddings.unsqueeze(0), 
    dim=2
)

# Visualizar matriz de similitud
plt.figure(figsize=(8, 6))
plt.imshow(similarity_matrix.cpu().numpy(), cmap='coolwarm', vmin=0.5, vmax=1.0)
plt.colorbar(label='Similitud Coseno')
plt.xticks(range(len(prompts)), prompts, rotation=45, ha='right')
plt.yticks(range(len(prompts)), prompts)
plt.title('Similitud entre Embeddings de Texto (CLIP)')
plt.tight_layout()
plt.show()

print("\nObserva cómo 'cat' y 'feline' tienen alta similitud, al igual que 'dog' y 'canine'.")

Cargando CLIP text encoder...


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

## 4. Componente Clave 2: Conditional U-Net

### Diferencia con U-Net Básica

La U-Net condicionada añade **cross-attention** para incorporar información del texto:

```
U-Net Básica:          U-Net Condicionada:
x_t → [Conv] → output  x_t → [Conv] → [Cross-Attention con texto] → output
                                ↑
                         Text embeddings
```

### Cross-Attention Mechanism

En cada capa de la U-Net:
1. **Query (Q)**: Viene de la imagen ruidosa
2. **Key (K) y Value (V)**: Vienen del texto embedding
3. **Attention**: $\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d}})V$

Esto permite que cada pixel "preste atención" a las palabras relevantes del prompt.

In [ ]:
# Visualizar arquitectura conceptual de Cross-Attention
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')

# Imagen con ruido (izquierda)
img_box = FancyBboxPatch((1, 6), 2, 2, boxstyle="round,pad=0.1",
                         facecolor='lightblue', edgecolor='blue', linewidth=2)
ax.add_patch(img_box)
ax.text(2, 7, 'Imagen\nRuidosa\n(x_t)', ha='center', va='center', fontsize=10, fontweight='bold')

# Text embedding (izquierda abajo)
text_box = FancyBboxPatch((1, 3), 2, 2, boxstyle="round,pad=0.1",
                          facecolor='lightgreen', edgecolor='green', linewidth=2)
ax.add_patch(text_box)
ax.text(2, 4, 'Text\nEmbedding', ha='center', va='center', fontsize=10, fontweight='bold')

# Query (desde imagen)
q_box = FancyBboxPatch((4.5, 6.5), 1.5, 1, boxstyle="round,pad=0.05",
                       facecolor='lightyellow', edgecolor='orange', linewidth=1.5)
ax.add_patch(q_box)
ax.text(5.25, 7, 'Query (Q)\nde imagen', ha='center', va='center', fontsize=9)

# Key y Value (desde texto)
k_box = FancyBboxPatch((4.5, 4.5), 1.5, 0.7, boxstyle="round,pad=0.05",
                       facecolor='lightcoral', edgecolor='red', linewidth=1.5)
ax.add_patch(k_box)
ax.text(5.25, 4.85, 'Key (K)', ha='center', va='center', fontsize=9)

v_box = FancyBboxPatch((4.5, 3.5), 1.5, 0.7, boxstyle="round,pad=0.05",
                       facecolor='lightcoral', edgecolor='red', linewidth=1.5)
ax.add_patch(v_box)
ax.text(5.25, 3.85, 'Value (V)', ha='center', va='center', fontsize=9)

# Cross-Attention
attn_box = FancyBboxPatch((7, 5), 2.5, 2, boxstyle="round,pad=0.1",
                          facecolor='lavender', edgecolor='purple', linewidth=2)
ax.add_patch(attn_box)
ax.text(8.25, 6, 'Cross-\nAttention', ha='center', va='center', fontsize=11, fontweight='bold')

# Output
out_box = FancyBboxPatch((10, 6), 1.5, 2, boxstyle="round,pad=0.1",
                         facecolor='lightgreen', edgecolor='green', linewidth=2)
ax.add_patch(out_box)
ax.text(10.75, 7, 'Features\nCondicionadas', ha='center', va='center', fontsize=10, fontweight='bold')

# Arrows
arrow1 = FancyArrowPatch((3, 7), (4.5, 7), arrowstyle='->', lw=2, color='blue')
arrow2 = FancyArrowPatch((3, 4), (4.5, 4.85), arrowstyle='->', lw=2, color='green')
arrow3 = FancyArrowPatch((3, 4), (4.5, 3.85), arrowstyle='->', lw=2, color='green')
arrow4 = FancyArrowPatch((6, 7), (7, 6.5), arrowstyle='->', lw=2, color='orange')
arrow5 = FancyArrowPatch((6, 4.85), (7, 6), arrowstyle='->', lw=2, color='red')
arrow6 = FancyArrowPatch((6, 3.85), (7, 5.5), arrowstyle='->', lw=2, color='red')
arrow7 = FancyArrowPatch((9.5, 6), (10, 7), arrowstyle='->', lw=2, color='purple')

for arrow in [arrow1, arrow2, arrow3, arrow4, arrow5, arrow6, arrow7]:
    ax.add_patch(arrow)

# Formula
ax.text(6, 2, r'$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d}})V$', 
        fontsize=12, ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.title('Arquitectura de Cross-Attention en U-Net Condicionada', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("La cross-attention permite que cada pixel de la imagen 'mire' las palabras relevantes del texto.")

## 5. Classifier-Free Guidance (CFG)

### Problema

Los modelos condicionados a veces ignoran parcialmente el texto y generan imágenes genéricas.

### Solución: Classifier-Free Guidance

Durante el sampling, calculamos dos predicciones:
1. **Conditional**: $\epsilon_\theta(x_t, c)$ (con el texto)
2. **Unconditional**: $\epsilon_\theta(x_t, \emptyset)$ (sin texto, prompt vacío)

Luego combinamos:
$$\hat{\epsilon}_\theta = \epsilon_\theta(x_t, \emptyset) + w \cdot (\epsilon_\theta(x_t, c) - \epsilon_\theta(x_t, \emptyset))$$

Donde:
- $w$ es el **guidance scale** (típicamente 7.5)
- $w > 1$: Sigue más el texto (más fidelidad, menos diversidad)
- $w = 1$: Sin guidance

### Intuición

```
Unconditional: "imagen cualquiera"
Conditional:   "imagen de un gato específico"
                        ↓
            Amplificamos la diferencia
                        ↓
           "SUPER gato específico"
```

In [ ]:
# Simulación conceptual de CFG
import numpy as np

# Simular predicciones de ruido (1D para simplicidad)
unconditional_noise = np.random.randn(100) * 0.5  # Predicción genérica
conditional_noise = np.random.randn(100) * 0.5 + np.sin(np.linspace(0, 4*np.pi, 100))  # Con patrón

# Aplicar CFG con diferentes guidance scales
guidance_scales = [1.0, 3.0, 7.5, 15.0]

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for idx, w in enumerate(guidance_scales):
    # Fórmula CFG
    guided_noise = unconditional_noise + w * (conditional_noise - unconditional_noise)
    
    axes[idx].plot(unconditional_noise, label='Unconditional', alpha=0.6, linestyle='--')
    axes[idx].plot(conditional_noise, label='Conditional', alpha=0.6, linestyle='--')
    axes[idx].plot(guided_noise, label=f'Guided (w={w})', linewidth=2)
    axes[idx].set_title(f'Guidance Scale = {w}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim(-5, 5)

plt.suptitle('Efecto del Classifier-Free Guidance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Nota: Con guidance scale alto (w=15), el patrón se amplifica mucho.")
print("En práctica, w=7.5 es un buen balance entre fidelidad y calidad.")

## 6. Latent Diffusion Models (Stable Diffusion)

### Problema con Pixel-Space Diffusion

Generar imágenes de alta resolución directamente es:
- **Muy lento**: 512x512x3 = 786,432 dimensiones
- **Costoso**: Requiere GPU de alta gama

### Solución: Latent Diffusion

Operar en un espacio comprimido:

```
Imagen (512×512×3) → [VAE Encoder] → Latent (64×64×4) → [Diffusion] → 
                                                                          ↓
Imagen generada ← [VAE Decoder] ← Latent denoised ←─────────────────────┘
```

**Ventajas:**
- 8× reducción en cada dimensión → 64× reducción en memoria
- Mucho más rápido
- Misma calidad visual

### Componentes de Stable Diffusion

1. **VAE (Variational Autoencoder)**: Comprime/descomprime imágenes
2. **CLIP Text Encoder**: Procesa el texto
3. **U-Net Condicionada**: Denoising en espacio latente
4. **Scheduler**: Controla el proceso de diffusion

## 7. Usar Stable Diffusion Pre-entrenado

Ahora usemos un modelo pre-entrenado para generar imágenes reales:

In [ ]:
# Cargar Stable Diffusion (versión más liviana)
print("Cargando Stable Diffusion...")
print("Esto puede tardar varios minutos la primera vez (descarga ~4GB)")

# Usar la versión 1.5 o 2.1
model_id = "runwayml/stable-diffusion-v1-5"  # Cambiar a "stabilityai/stable-diffusion-2-1" si lo deseas

# Cargar pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32,
    safety_checker=None  # Desactivar para experimentación
)
pipe = pipe.to(device)

# Si tienes GPU, habilitar optimizaciones
if device.type == 'cuda':
    pipe.enable_attention_slicing()  # Reduce uso de memoria
    # pipe.enable_xformers_memory_efficient_attention()  # Requiere xformers instalado

print("Modelo cargado exitosamente!")

Cargando Stable Diffusion...
Esto puede tardar varios minutos la primera vez (descarga ~4GB)


In [ ]:
# Función para generar imágenes
def generate_image(prompt, negative_prompt="", guidance_scale=7.5, num_inference_steps=50, seed=None):
    """
    Genera una imagen desde un prompt de texto
    
    Args:
        prompt: Descripción de la imagen a generar
        negative_prompt: Qué NO debe aparecer en la imagen
        guidance_scale: Qué tan fuerte seguir el prompt (1-20, típicamente 7.5)
        num_inference_steps: Número de pasos de denoising (más = mejor calidad pero más lento)
        seed: Semilla para reproducibilidad
    
    Returns:
        Imagen generada (PIL Image)
    """
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    with torch.autocast(device.type if device.type == 'cuda' else 'cpu'):
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            generator=generator
        ).images[0]
    
    return image


# Ejemplos de prompts
prompts = [
    "A photo of a cute cat wearing a wizard hat, studio lighting, detailed, 4k",
    "A beautiful landscape with mountains and a lake at sunset, oil painting style",
    "A futuristic city with flying cars, cyberpunk, neon lights, highly detailed",
    "A portrait of a robot reading a book in a library, digital art"
]

# Generar imágenes
print("Generando imágenes...\n")
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
axes = axes.flatten()

for idx, prompt in enumerate(prompts):
    print(f"[{idx+1}/4] Generando: {prompt[:60]}...")
    
    image = generate_image(
        prompt=prompt,
        negative_prompt="blurry, bad quality, ugly, distorted",
        guidance_scale=7.5,
        num_inference_steps=30,  # Reducido para velocidad, usar 50 para mejor calidad
        seed=42 + idx  # Para reproducibilidad
    )
    
    axes[idx].imshow(image)
    axes[idx].set_title(prompt[:50] + '...', fontsize=9)
    axes[idx].axis('off')

plt.suptitle('Imágenes Generadas con Stable Diffusion', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nGeneración completa!")

## 8. Experimentos: Efecto de los Parámetros

### 8.1 Guidance Scale

In [ ]:
# Comparar diferentes guidance scales
prompt = "A photograph of an astronaut riding a horse on mars, highly detailed"
guidance_scales = [1.0, 5.0, 7.5, 15.0]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, scale in enumerate(guidance_scales):
    print(f"Generando con guidance_scale={scale}...")
    
    image = generate_image(
        prompt=prompt,
        guidance_scale=scale,
        num_inference_steps=30,
        seed=123
    )
    
    axes[idx].imshow(image)
    axes[idx].set_title(f'Guidance Scale = {scale}', fontsize=11)
    axes[idx].axis('off')

plt.suptitle(f'Prompt: "{prompt}"', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObservaciones:")
print("- scale=1.0: Ignora mucho el prompt, imagen más creativa pero menos precisa")
print("- scale=7.5: Balance óptimo entre fidelidad y calidad")
print("- scale=15.0: Sigue el prompt muy de cerca, puede ser sobre-saturado")

### 8.2 Number of Inference Steps

In [ ]:
# Comparar diferentes números de steps
prompt = "A serene zen garden with cherry blossoms, Japanese style, peaceful"
steps_list = [10, 20, 30, 50]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, steps in enumerate(steps_list):
    print(f"Generando con {steps} steps...")
    
    image = generate_image(
        prompt=prompt,
        guidance_scale=7.5,
        num_inference_steps=steps,
        seed=456
    )
    
    axes[idx].imshow(image)
    axes[idx].set_title(f'{steps} Steps', fontsize=11)
    axes[idx].axis('off')

plt.suptitle(f'Prompt: "{prompt}"', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObservaciones:")
print("- 10 steps: Rápido pero puede tener artefactos")
print("- 30 steps: Buen balance velocidad/calidad")
print("- 50 steps: Máxima calidad, más lento")

### 8.3 Negative Prompts

In [ ]:
# Comparar con y sin negative prompt
prompt = "A portrait of a person"

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Sin negative prompt
print("Generando sin negative prompt...")
image1 = generate_image(
    prompt=prompt,
    negative_prompt="",
    seed=789
)
axes[0].imshow(image1)
axes[0].set_title('Sin Negative Prompt', fontsize=12)
axes[0].axis('off')

# Con negative prompt
print("Generando con negative prompt...")
image2 = generate_image(
    prompt=prompt,
    negative_prompt="blurry, bad anatomy, deformed, ugly, low quality, pixelated",
    seed=789
)
axes[1].imshow(image2)
axes[1].set_title('Con Negative Prompt', fontsize=12)
axes[1].axis('off')

plt.suptitle(f'Prompt: "{prompt}"', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nEl negative prompt ayuda a evitar características no deseadas.")

## 9. Técnicas Avanzadas de Prompting

### 9.1 Estructura de un Buen Prompt

```
[Sujeto] + [Estilo] + [Detalles] + [Iluminación] + [Calidad]
```

Ejemplos:
- ✅ **Bueno**: "A majestic lion, digital art, detailed fur, golden hour lighting, 4k, trending on artstation"
- ❌ **Malo**: "lion"

### 9.2 Palabras Clave Útiles

**Calidad:**
- highly detailed, 4k, 8k, sharp focus, masterpiece
- trending on artstation, award winning

**Estilo:**
- oil painting, watercolor, digital art, photograph, 3D render
- cyberpunk, fantasy, realistic, cartoon, anime

**Iluminación:**
- studio lighting, golden hour, dramatic lighting, soft lighting
- volumetric lighting, rim lighting

**Artistas (para estilo):**
- by Greg Rutkowski, by Artgerm, by Studio Ghibli

In [ ]:
# Comparar prompts simples vs detallados
prompts_comparison = [
    ("A dragon", "Simple"),
    ("A majestic dragon flying over mountains, fantasy art, highly detailed scales, "
     "dramatic lighting, volumetric clouds, 4k, trending on artstation", "Detallado")
]

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

for idx, (prompt, label) in enumerate(prompts_comparison):
    print(f"Generando prompt {label}...")
    
    image = generate_image(
        prompt=prompt,
        negative_prompt="blurry, bad quality",
        seed=999
    )
    
    axes[idx].imshow(image)
    axes[idx].set_title(f'{label}\n"{prompt[:60]}..."', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('Impacto de Prompts Detallados', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Comparación: DALL-E 2 vs Stable Diffusion vs Imagen

### Arquitecturas

| Modelo | Text Encoder | Diffusion Space | Características |
|--------|--------------|-----------------|------------------|
| **DALL-E 2** | CLIP | Pixel + Latent (GLIDE) | Usa prior diffusion para texto→imagen |
| **Stable Diffusion** | CLIP | Latent (VAE) | Open source, eficiente |
| **Imagen** | T5 | Pixel | Cascading diffusion (64→256→1024) |

### DALL-E 2 Architecture

```
Texto → [CLIP Text] → Text Embedding → [Prior (Diffusion)] → Image Embedding
                                                                      ↓
                                              [Decoder (Diffusion)] → Imagen
```

Dos etapas:
1. **Prior**: Convierte texto en CLIP image embedding
2. **Decoder**: Convierte image embedding en imagen

### Imagen Architecture

```
Texto → [T5] → Embeddings → [Diffusion 64×64] → [Super-res 256] → [Super-res 1024]
```

Cascading diffusion:
1. Base model: 64×64
2. Super-resolution 1: 64→256
3. Super-resolution 2: 256→1024

### ¿Cuál es mejor?

- **DALL-E 2**: Mejor comprensión de texto complejo, creative composition
- **Stable Diffusion**: Open source, más controlable, eficiente
- **Imagen**: Mejor photorealism, texto en imágenes

## 11. Implementación Simplificada: Conditional Diffusion

Implementemos un modelo de difusión condicionado simple para entender mejor:

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleConditionalUNet(nn.Module):
    """U-Net simple con condicionamiento de texto"""
    
    def __init__(self, in_channels=1, out_channels=1, text_embed_dim=512, time_embed_dim=128):
        super().__init__()
        
        # Time embedding
        self.time_embed = nn.Sequential(
            nn.Linear(time_embed_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim)
        )
        
        # Text projection (para cross-attention simplificada)
        self.text_proj = nn.Linear(text_embed_dim, 256)
        
        # Encoder
        self.conv1 = nn.Conv2d(in_channels, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        
        # Cross-attention simplificada (solo en el bottleneck)
        self.cross_attn = SimpleCrossAttention(256, 256)
        
        # Decoder
        self.upconv1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv4 = nn.Conv2d(256, 128, 3, padding=1)  # 256 por skip connection
        
        self.upconv2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv5 = nn.Conv2d(128, 64, 3, padding=1)
        
        self.out = nn.Conv2d(64, out_channels, 1)
    
    def forward(self, x, t, text_embed):
        """
        Args:
            x: Imagen con ruido [B, C, H, W]
            t: Time embeddings [B, time_embed_dim]
            text_embed: Text embeddings [B, text_embed_dim]
        """
        # Process time
        t_emb = self.time_embed(t)  # [B, time_embed_dim]
        
        # Process text
        text_ctx = self.text_proj(text_embed)  # [B, 256]
        
        # Encoder
        x1 = F.silu(self.conv1(x))
        x2 = F.silu(self.conv2(F.max_pool2d(x1, 2)))
        x3 = F.silu(self.conv3(F.max_pool2d(x2, 2)))
        
        # Cross-attention con texto
        x3 = self.cross_attn(x3, text_ctx.unsqueeze(1))  # Add seq dimension
        
        # Decoder
        x = self.upconv1(x3)
        x = torch.cat([x, x2], dim=1)
        x = F.silu(self.conv4(x))
        
        x = self.upconv2(x)
        x = torch.cat([x, x1], dim=1)
        x = F.silu(self.conv5(x))
        
        return self.out(x)


class SimpleCrossAttention(nn.Module):
    """Cross-attention simplificada"""
    
    def __init__(self, query_dim, context_dim):
        super().__init__()
        self.to_q = nn.Linear(query_dim, query_dim)
        self.to_k = nn.Linear(context_dim, query_dim)
        self.to_v = nn.Linear(context_dim, query_dim)
        self.scale = query_dim ** -0.5
    
    def forward(self, x, context):
        """
        Args:
            x: Features [B, C, H, W]
            context: Text context [B, seq_len, context_dim]
        """
        B, C, H, W = x.shape
        
        # Reshape x para attention: [B, H*W, C]
        x_flat = x.view(B, C, H*W).permute(0, 2, 1)
        
        # Compute Q, K, V
        q = self.to_q(x_flat)  # [B, H*W, C]
        k = self.to_k(context)  # [B, seq_len, C]
        v = self.to_v(context)  # [B, seq_len, C]
        
        # Attention
        attn = torch.matmul(q, k.transpose(-1, -2)) * self.scale  # [B, H*W, seq_len]
        attn = F.softmax(attn, dim=-1)
        
        # Apply attention
        out = torch.matmul(attn, v)  # [B, H*W, C]
        
        # Reshape back
        out = out.permute(0, 2, 1).view(B, C, H, W)
        
        # Residual connection
        return x + out


# Crear modelo de ejemplo
conditional_model = SimpleConditionalUNet()
print(f"Modelo condicional creado con {sum(p.numel() for p in conditional_model.parameters())} parámetros")

# Test forward pass
x_test = torch.randn(2, 1, 28, 28)
t_test = torch.randn(2, 128)
text_test = torch.randn(2, 512)

output = conditional_model(x_test, t_test, text_test)
print(f"Input shape: {x_test.shape}")
print(f"Output shape: {output.shape}")
print("Forward pass exitoso!")

## 12. Resumen y Conceptos Clave

### ¿Qué aprendimos?

1. **Text Encoding**: CLIP convierte texto en embeddings semánticos
   - Captura el significado del prompt
   - Permite guiar la generación

2. **Conditional U-Net**: Cross-attention integra texto en denoising
   - Query: de la imagen
   - Key/Value: del texto
   - Cada pixel "mira" las palabras relevantes

3. **Classifier-Free Guidance**: Amplifica el efecto del texto
   - Compara conditional vs unconditional
   - Guidance scale controla fidelidad vs creatividad

4. **Latent Diffusion**: Opera en espacio comprimido
   - VAE encoder/decoder
   - 64× más eficiente que pixel space
   - Stable Diffusion usa este enfoque

### Comparación de Modelos

- **DALL-E 2**: Prior + Decoder, best text understanding
- **Stable Diffusion**: Latent diffusion, open source, efficient
- **Imagen**: Cascading diffusion, best photorealism

### Mejores Prácticas de Prompting

1. Ser descriptivo y específico
2. Incluir estilo artístico
3. Especificar iluminación y calidad
4. Usar negative prompts para evitar defectos
5. Experimentar con guidance scale (7.5 es un buen inicio)

### Limitaciones

- Dificultad con texto en imágenes
- A veces ignora parte del prompt
- Puede generar anatomías incorrectas
- Bias del dataset de entrenamiento

### Direcciones Futuras

- **Video diffusion**: Generación de videos (Imagen Video, Runway)
- **3D diffusion**: Generar modelos 3D (DreamFusion, Magic3D)
- **Fast sampling**: Reducir pasos de inferencia (LCM, SDXL Turbo)
- **Better control**: ControlNet, IP-Adapter

## 13. Ejercicios Prácticos

### Ejercicio 1: Experimentar con Prompts

Genera imágenes con diferentes prompts y compara:
- Mismo sujeto, diferentes estilos
- Misma escena, diferentes iluminaciones

### Ejercicio 2: Optimizar Parámetros

Para un prompt específico, encuentra:
- El mejor guidance_scale
- El mínimo número de steps para buena calidad

### Ejercicio 3: Negative Prompts

Experimenta con diferentes negative prompts:
- Genera retratos con/sin negative prompts para anatomía
- Compara la calidad

### Ejercicio 4: Implementar Training Loop

Extiende el `SimpleConditionalUNet` para:
- Entrenar en MNIST condicionado por dígitos
- Implementar classifier-free guidance

### Ejercicio 5: Explorar ControlNet

Investiga y prueba ControlNet para:
- Generar imágenes condicionadas por edges
- Controlar poses humanas

## Referencias

### Papers Fundamentales

- [CLIP (Radford et al., 2021)](https://arxiv.org/abs/2103.00020)
- [DALL-E 2 (Ramesh et al., 2022)](https://arxiv.org/abs/2204.06125)
- [Imagen (Saharia et al., 2022)](https://arxiv.org/abs/2205.11487)
- [Latent Diffusion / Stable Diffusion (Rombach et al., 2022)](https://arxiv.org/abs/2112.10752)
- [Classifier-Free Guidance (Ho & Salimans, 2022)](https://arxiv.org/abs/2207.12598)

### Recursos Adicionales

- [Stable Diffusion GitHub](https://github.com/Stability-AI/stablediffusion)
- [Hugging Face Diffusers](https://huggingface.co/docs/diffusers/index)
- [The Illustrated Stable Diffusion](https://jalammar.github.io/illustrated-stable-diffusion/)
- [ControlNet Paper](https://arxiv.org/abs/2302.05543)